In [12]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments

In [20]:
from datasets import Dataset

In [2]:
df  = pd.read_csv('~/Documents/TarifaBot_GenAI/dataset_tarifas_bancarias_v2.csv')

In [3]:
df.head()

,Serviço,Tarifa,Isenção,Detalhes,Resumo
0,Conta digital,"Tarifa mensal de R$20,00",Isento para contas universitárias,Taxas reduzidas para servidores públicos.,O serviço 'Conta digital' possui uma tarifa de...
1,Investimento CDB,Gratuito,Desconto para clientes do banco,Compensação em até 1 dia útil.,O serviço 'Investimento CDB' possui uma tarifa...
2,Financiamento imobiliário,"R$10,00 por operação",Sem restrições,Ideal para quem busca praticidade.,O serviço 'Financiamento imobiliário' possui u...
3,Saque na rede 24h,"Taxa de 0,5% ao mês","Isento para gastos acima de R$5.000,00/mês",Taxas reduzidas para servidores públicos.,O serviço 'Saque na rede 24h' possui uma tarif...
4,Cartão de crédito,"R$5,00 por operação",Isento para clientes premium,Oferece benefícios exclusivos para clientes do...,O serviço 'Cartão de crédito' possui uma tarif...


In [4]:
len(df)

200

In [5]:
df.isnull().sum()

Serviço     0
Tarifa      0
Isenção     0
Detalhes    0
Resumo      0
dtype: int64

In [6]:
df.describe()

,Serviço,Tarifa,Isenção,Detalhes,Resumo
count,200,200,200,200,200
unique,16,7,5,7,194
top,Conta digital,Gratuito,Desconto para clientes do banco,Taxas reduzidas para servidores públicos.,O serviço 'Pix' possui uma tarifa de Taxa de 0...
freq,19,36,43,32,2


In [7]:
#sepração em treino e teste

x_train, x_test = train_test_split(df, test_size= 0.2, random_state= 0)

In [8]:
len(x_train)

160

Tokenização: significa que vai transformar os dados de texto em numéricos (Ex:"Olá, tudo bem?" = [14792, 11, 1714, 703] )
Cada modelo possui uma tokenização própria pré-treinada. No case em questão usaremos o GPT2.

In [9]:
#carregar tokenizador
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

#carrega o modelo GPT2
model = GPT2LMHeadModel.from_pretrained("gpt2")

Pad_token = utilizado para preencher "espaços" em frases curtas, mas que elas tenham o mesmo comprimento das demais. No case trabalho o pad_token utilizado será eos_token('<|endoftext|>') geralmente usado para sinalizar fim da sequencia de texto no GPT2, que não tem padding token por padrão e por isso estamos definindo.
A utilizade é poder processar as frases em batch (lote), conjunto de exemplos processados simultaneamente durante o treinamento ou inferência de um modelo de machine learning.

In [10]:
#definindo padding token
tokenizer.pad_token = tokenizer.eos_token

In [19]:
# Tokenizando o prompt e resposta
def tokenize_function(examples):
    prompts = [f"Qual a tarifa do serviço '{row['Serviço']}'?" for row in examples['Serviço']]
    respostas = examples['Resumo']
    
    # Tokenizando os prompts e as respostas
    inputs = tokenizer(prompts, padding=True, truncation=True, max_length=512)
    labels = tokenizer(respostas, padding=True, truncation=True, max_length=512)
    
    # Adicionando as labels no dataset
    inputs['labels'] = labels['input_ids']
    
    return inputs

In [21]:
# Suponha que x_train seja um DataFrame com as colunas 'Serviço' e 'Resumo'
train_dataset = Dataset.from_pandas(x_train)

In [22]:
# Aplicando a tokenização ao dataset
train_dataset = train_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/160 [00:00<?, ? examples/s]


TypeError: string indices must be integers, not 'str'

In [13]:
# Definindo parametros de treinamento

training_args = TrainingArguments(
    output_dir='./results',          # Diretório de saída para os resultados
    num_train_epochs=3,              # Número de épocas de treinamento
    per_device_train_batch_size=8,   # Tamanho do lote de treinamento por dispositivo
    per_device_eval_batch_size=8,    # Tamanho do lote de avaliação por dispositivo
    warmup_steps=500,                # Número de passos para aquecer
    weight_decay=0.01,               # Regularização para evitar overfitting
    logging_dir='./logs',            # Diretório de logs
    logging_steps=10,                # Log a cada 10 passos
    save_steps=500,                  # Salvar o modelo a cada 500 passos
)

In [14]:
#criando o trainer

trainer = Trainer( model= model, args= training_args, train_dataset= x_train, eval_dataset=x_test)

In [18]:
#iniciando o treinamento
trainer.train()

KeyError: 102